<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Authenticate with Google Cloud


In [ ]:
from google.colab import auth
auth.authenticate_user()

# Step 2: Install Spark and BigQuery connector

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirror.digitalpacific.com.au/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

# Step 3: Set environment variables for Spark and Java

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# Step 4: Install BigQuery Connector for Spark

In [ ]:
!wget -q https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.29.0.jar


# Step 5: Initialize Spark session

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GCS to BigQuery Streaming with Auto-Table Creation") \
    .config("spark.jars", "/content/spark-bigquery-with-dependencies_2.12-0.29.0.jar") \
    .getOrCreate()

# Step 6: Define GCS input path and output BigQuery table

In [ ]:
input_path = "gs://edit-de-project-streaming-data/carris-vehicles"
output_table = "data-eng-dev-437916.data_eng_project_group3.raw_vehicles"

# Step 7: Read streaming data from GCS

In [ ]:
streaming_df = spark.readStream \
    .format("json") \
    .load(input_path)

# Step 8: Write streaming data to BigQuery with auto-table creation

In [ ]:
streaming_query = streaming_df.writeStream \
    .format("bigquery") \
    .option("table", output_table) \
    .option("checkpointLocation", "gs://edit-data-eng-project-group3/streaming_data/checkpoints") \
    .option("temporaryGcsBucket", "gs://edit-data-eng-project-group3/streaming_data/temp-bucket") \
    .option("writeDisposition", "WRITE_APPEND") \
    .option("createDisposition", "CREATE_IF_NEEDED") \
    .outputMode("append") \
    .start()

# Step 9: Wait for the streaming query to finish

In [ ]:
streaming_query.awaitTermination()